# GraphRAG Benchmarking

This notebook benchmarks our GraphRAG implementation using the Origin of Covid-19 dataset.

## Configuration Options

### Document Processing
- chunk_size: Number of words per chunk (default: 500)
- chunk_overlap: Number of overlapping words between chunks (default: 50)
- enable_chunking: Whether to split documents into chunks (default: True)

### Graph Construction
- min_entity_freq: Minimum frequency for entity inclusion (default: 2)
- max_relation_distance: Maximum token distance for relationships (default: 10)

### Hybrid Search
- k_graph: Number of graph-based results (default: 5)
- k_vector: Number of vector-based results (default: 3)
- alpha: Weight for combining scores (default: 0.7)
- search_type: Type of vector search ('script' or 'knn', default: 'script')
- similarity_threshold: Minimum similarity score (default: None)

### API Settings
- max_retries: Maximum retry attempts (default: 5)
- min_delay: Minimum retry delay in seconds (default: 1)
- max_delay: Maximum retry delay in seconds (default: 60)

In [ ]:
import os
import json
import sys
import boto3
from pathlib import Path
from typing import List, Dict, Any, Optional
from tqdm.auto import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

# Add project root to path for imports
project_root = Path("../..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import utilities
from utils.metrics.rag_metrics import RAGMetricsEvaluator
from utils.notebook_utils.dataset_utils import (
    load_labeled_dataset,
    examine_dataset_structure,
    save_dataset_info,
    convert_to_ragas_dataset
)
from utils.notebook_utils.importable import notebook_to_module
from utils.aws.opensearch_utils import OpenSearchManager

# Import graph RAG components
from rag_implementations.graph_rag.components.benchmark import run_evaluation, save_results

In [ ]:
# Dataset Configuration
DATASET_NAME = "OriginOfCovid19Dataset"
DATASET_DIR = project_root / "datasets/rag_evaluation/labeled/covid19_origin"
NUM_EVAL_SAMPLES = None  # Set to a number for partial evaluation

# Neptune Configuration
NEPTUNE_CONFIG = {
    "cluster_name": "test-graph-rag-benchmark"
}

# OpenSearch Configuration
OPENSEARCH_CONFIG = {
    "domain_name": "graph-rag-benchmark-store",
    "cleanup_enabled": True,  # Clean up domain after benchmarking
    "verbose": True          # Show detailed progress
}

# RAG Configuration
RAG_CONFIG = {
    # Document processing
    "chunk_size": 500,  # 500 words ≈ 2000 chars
    "chunk_overlap": 50,  # 50 words overlap
    "enable_chunking": True,
    
    # Graph construction
    "min_entity_freq": 2,
    "max_relation_distance": 10,
    
    # Hybrid search
    "k_graph": 5,
    "k_vector": 3,
    "alpha": 0.7,
    "search_type": "script",
    "similarity_threshold": None,
    
    # OpenSearch config
    "index_settings": None,
    
    # API settings
    "max_retries": 5,
    "min_delay": 1.0,
    "max_delay": 60.0
}

print("Note: This notebook uses Amazon Neptune and OpenSearch which incur costs.")
print("Neptune and OpenSearch resources will be cleaned up after benchmarking.")
print("Set cleanup_enabled = False if you want to preserve the resources.")

In [ ]:
# Import implementations
implementation_path = str(project_root / 'rag_implementations/graph_rag/implementation.ipynb')
ingestion_path = str(project_root / 'rag_implementations/graph_rag/ingestion.ipynb')
GraphRAG = notebook_to_module(implementation_path).GraphRAG
ingest_documents = notebook_to_module(ingestion_path).ingest_documents

# Initialize evaluator
evaluator = RAGMetricsEvaluator()

In [ ]:
# Load and examine dataset
print(f"Loading {DATASET_NAME}...")
dataset, documents = load_labeled_dataset(DATASET_DIR, download_if_missing=True)
print(f"Loaded {len(dataset.examples)} examples and {len(documents)} documents")

# Get evaluation examples
eval_examples = dataset.examples[:NUM_EVAL_SAMPLES] if NUM_EVAL_SAMPLES else dataset.examples
print(f"Using {len(eval_examples)} examples for evaluation")

# Examine dataset structure
dataset_info = examine_dataset_structure(dataset, documents)
print("\nDataset Structure:")
print(json.dumps(dataset_info, indent=2))

# Save dataset info
save_dataset_info(dataset_info, DATASET_DIR / 'dataset_info.json')
print(f"\nDataset information saved to: {DATASET_DIR / 'dataset_info.json'}")

In [ ]:
# Set up OpenSearch domain and get endpoint
print("Setting up OpenSearch domain...")
opensearch_manager = OpenSearchManager(**OPENSEARCH_CONFIG)
endpoint = opensearch_manager.setup_domain()

# Set endpoint in environment for vector store
os.environ['OPENSEARCH_HOST'] = endpoint

# Initialize GraphRAG with configuration
print("\nInitializing GraphRAG...")
rag = None
try:
    rag = GraphRAG(
        index_name=f"{DATASET_NAME.lower()}-benchmark",
        graph_store_config=NEPTUNE_CONFIG,
        **RAG_CONFIG
    )
    
    # Check if documents already exist
    print("\nChecking existing documents...")
    doc_count = rag.vector_store.opensearch_client.client.count(index=rag.index_name)['count']
    if doc_count > 0:
        print(f"Found {doc_count} documents already indexed")
        print("Skipping ingestion to avoid duplicates")
    else:
        print("Ingesting documents...")
        source_dir = DATASET_DIR / "source_files"
        ingest_documents(
            str(source_dir),
            rag,
            metadata={'dataset': DATASET_NAME},
            batch_size=100
        )
        
    # Print graph store status
    print("\nGraph Store Status:")
    if rag.graph_store and rag.graph_store._initialized:
        print("✅ Neptune connection successful")
        print(f"Endpoint: {rag.graph_store.endpoint}")
    else:
        print("❌ Neptune connection not initialized")
        
except Exception as e:
    print(f"\nError during initialization: {str(e)}")
    if rag:
        # Clean up on initialization failure, but don't delete resources
        rag.cleanup(delete_resources=False)
    raise

In [ ]:
# Run evaluation and save results
results_dir = project_root / "evaluation_pipelines/rag_evaluations/results"
results_dir.mkdir(exist_ok=True)

results = run_evaluation(rag, dataset, evaluator, eval_examples)
save_results(results, DATASET_NAME, results_dir)

In [ ]:
# Example queries
example_queries = [
    "What is the main focus of the article 'The Origin of COVID-19 and Why It Matters'?",
    "What evidence suggests that SARS-CoV-2 emerged naturally rather than being engineered?",
    "What are some potential consequences of not understanding how COVID-19 emerged?"
]

print("Testing example queries...\n")
for query in example_queries:
    print(f"Query: {query}")
    result = rag.query(query)
    
    print("\nGraph Context:")
    for ctx in result['graph_context']:
        print(f"\nDocument {ctx['doc_id']}:")
        print("Entities:", ", ".join([f"{e['text']} ({e['label']})" for e in ctx['entities']]))
        print("Relations:", ", ".join([f"{r['from']} {r['label']} {r['to']}" for r in ctx['relations']]))
    
    print(f"\nResponse: {result['response']}\n")
    print("-" * 80)

In [ ]:
# Resource Cleanup
if OPENSEARCH_CONFIG['cleanup_enabled']:
    print("=== Cleaning Up Resources ===")
    print("Warning: This will delete all resources and indexed data")
    print("This may take 15-20 minutes to complete")
    try:
        if rag:
            # Delete resources and wait for deletion to complete
            rag.cleanup(delete_resources=True)
            
            # Wait for OpenSearch domain deletion
            if opensearch_manager:
                opensearch_manager._wait_for_deletion()
            
            # Wait for Neptune deletion
            if rag.graph_store and rag.graph_store.neptune_manager:
                rag.graph_store.neptune_manager.cleanup()
                
            print("✅ Cleanup successful")
    except Exception as e:
        print(f"❌ Error during cleanup: {str(e)}")
        print("Some resources may need to be cleaned up manually")